In [3]:
from anthropic import Anthropic


In [2]:
import os

my_api_key = os.getenv("ANTHROPIC_API_KEY")

In [4]:
client = Anthropic(
    api_key=my_api_key
)

In [5]:
our_first_message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about a pet chicken"}
    ]
)

print(our_first_message.content[0].text)

Here is a haiku about a pet chicken:

Feathered friend clucks near,
Pecking seeds from the garden,
Loyal companion.
